In [10]:
from gensim.models import KeyedVectors
from tqdm import tqdm
import numpy as np

In [18]:
glove = '/home/manni/embs/gloveW2V.6B.300d.txt'
w2v = '/home/manni/embs/w2v.bin.gz'
wiki2v = '/home/manni/embs/enwiki_20180420_300d.txt.bz2'


In [28]:
glove_model = KeyedVectors.load_word2vec_format(glove, binary=False)

In [19]:
w2v_model = KeyedVectors.load_word2vec_format(w2v, binary=True)

In [ ]:
wiki_model = KeyedVectors.load_word2vec_format(wiki2v, binary=False)

In [ ]:
words = wiki_model.most_similar('bank',topn=100)

In [ ]:
wiki_model.most_similar('bank',topn=100)

In [20]:
#w2v_model.vocab

In [66]:
def total_overlap(word,model,topn=100):
    overlap_matrix = np.zeros((topn,topn),dtype=np.int8)
    words = model.most_similar(word,topn=topn)
    words = set([i[0] for i in words])
    nns = dict()
    for word in words:
        _words = model.most_similar(word,topn=5)
        _words = set([i[0] for i in _words])
        nns[word]=_words
    for i,(k,a) in enumerate(nns.items()):
        for j,(kk,b) in enumerate(nns.items()):
            if k == kk:
                overlap_matrix[i][j]=1
                continue
            overlap = len(a & b)
            overlap_matrix[i][j]=overlap
    return nns,overlap_matrix

In [67]:
nns,ovm = total_overlap('bank',glove_model,topn=100)

In [ ]:
def print_closer(n):
    print(voc[n])
    print(':::::')
    nn = set()
    for i,val in enumerate(ovm[n]>0):
        if val:
            print(i,voc[i])
            nn.add(voc[i])
    return nn

In [ ]:
a = print_closer(3)
b = print_closer(15)
print(a&b)

In [68]:
clusters = dict()
voc = list(nns.keys())
for v in voc:
    clusters[v]=-1
c = 0
for i in range(len(ovm)):
    for j in range(len(ovm)):
        if i == j:
            continue
        vec = ovm[i]
        vec = vec>0
        vec_ = ovm[j]
        vec_= vec_>0
        list_a = [voc[i_] for i_,x in enumerate(vec) if x]
        list_b = [voc[i_] for i_,x in enumerate(vec_) if x]
        overlap = len(set(list_a) & set(list_b))
        if overlap>3:
            #print(i,':',voc[i],j,':',voc[j])
            #if i == 10 or j == 10:
            #    print(i,j,clusters[voc[i]],clusters[voc[j]])
            if clusters[voc[i]]>-1 and clusters[voc[j]]>-1:
                c_ = min(clusters[voc[i]],clusters[voc[j]])
                clusters[voc[i]]=c_
                clusters[voc[j]]=c_
            elif clusters[voc[i]]>-1:
                clusters[voc[j]]=clusters[voc[i]]
            elif clusters[voc[j]]>-1:
                clusters[voc[i]]=clusters[voc[j]]
            else:
                clusters[voc[i]]=c
                clusters[voc[j]]=c
                c+=1
            #if i == 10 or j == 10:
            #    print(i,j,clusters[voc[i]],clusters[voc[j]])
        #if np.array_equal(vec,vec_):
        #    print(i,':',voc[i],j,':',voc[j])
    #print('***')

In [63]:
clusters

{'central': -1,
 'hsbc': 0,
 'markets': -1,
 'branch': -1,
 'securities': -1,
 'money': 1,
 'lenders': 1,
 'palestinian': -1,
 'assets': 1,
 'commercial': -1,
 'deposits': 1,
 'amro': 3,
 'settlements': -1,
 'institution': -1,
 'subsidiary': -1,
 'financing': 1,
 'banco': -1,
 'finance': -1,
 'banks': 1,
 'settlement': -1,
 'swiss': -1,
 'institutions': -1,
 'ramallah': -1,
 'occupied': -1,
 'month': -1,
 'currency': -1,
 'debt': 1,
 'earlier': -1,
 'stake': -1,
 'mortgage': 1,
 'raised': -1,
 'banking': 1,
 'rates': -1,
 'liquidity': 1,
 'interbk': -1,
 'ecb': -1,
 'treasury': -1,
 'dollars': -1,
 'fed': -1,
 'cash': 1,
 'investments': 1,
 'government': -1,
 'banker': -1,
 'citibank': 0,
 'raise': -1,
 'imf': -1,
 'offices': -1,
 'brokerage': -1,
 'monetary': -1,
 'the': -1,
 'deposit': -1,
 'asset': 1,
 'fargo': -1,
 'sector': -1,
 'borrowing': -1,
 'economist': -1,
 'bankers': 1,
 'dresdner': -1,
 'barclays': -1,
 'accounts': -1,
 'withdrawals': -1,
 'investment': 1,
 'lender': 1,
 

In [69]:
clusters_ = dict()
for i,c in enumerate(clusters.values()):
    if c == -1:
        continue
    if c in clusters_:
        clusters_[c].append(voc[i])
    else:
        clusters_[c] = [voc[i]]

In [70]:
clusters_

{0: ['hsbc',
  'amro',
  'citibank',
  'fargo',
  'economist',
  'dresdner',
  'barclays',
  'deutsche',
  'abn',
  'suisse',
  'ubs',
  'chase',
  'jpmorgan',
  'citigroup',
  'sachs'],
 1: ['securities',
  'money',
  'lenders',
  'assets',
  'deposits',
  'financing',
  'finance',
  'banks',
  'institutions',
  'debt',
  'mortgage',
  'banking',
  'rates',
  'liquidity',
  'fed',
  'cash',
  'investments',
  'raise',
  'deposit',
  'asset',
  'borrowing',
  'bankers',
  'accounts',
  'investment',
  'lender',
  'holdings',
  'loan',
  'funds',
  'capital',
  'interest',
  'loans',
  'lending',
  'rate',
  'reserve',
  'fund',
  'trust',
  'financial',
  'credit'],
 3: ['palestinian', 'ramallah', 'israel', 'israeli', 'palestinians', 'gaza']}

In [ ]:
def get_matrix(model,vocab):
    matrix = list()
    for word in vocab:
        vec = model.get_vector(word)
        matrix.append(vec)
    return matrix

In [ ]:
matrix = get_matrix(glove_model,vocab)

In [ ]:
matrix = np.asarray(matrix)

# clustering

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
from collections import Counter

In [ ]:
db = DBSCAN(eps=0.4,min_samples=2,metric='cosine').fit(matrix)

In [ ]:
labels = db.labels_
clusters = dict()
for i,c in enumerate(labels):
    if c == -1:
        continue
    if c in clusters:
        clusters[c].append(vocab[i])
    else:
        clusters[c] = [vocab[i]]

In [ ]:
clusters

In [ ]:
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

In [ ]:
X = [[1, 2], [2, 2], [2, 3],[8, 7], [8, 8], [25, 80]]
clustering = DBSCAN(eps=3, min_samples=2).fit(X)
clustering.labels_

# wordnet

In [168]:
from nltk.corpus import wordnet as wn
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from collections import Counter

In [75]:
wn.synsets('dog')

[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01'),
 Synset('chase.v.01')]

In [140]:
wn.synsets('dog')[0].hyponyms()

[Synset('basenji.n.01'),
 Synset('corgi.n.01'),
 Synset('cur.n.01'),
 Synset('dalmatian.n.02'),
 Synset('great_pyrenees.n.01'),
 Synset('griffon.n.02'),
 Synset('hunting_dog.n.01'),
 Synset('lapdog.n.01'),
 Synset('leonberg.n.01'),
 Synset('mexican_hairless.n.01'),
 Synset('newfoundland.n.01'),
 Synset('pooch.n.01'),
 Synset('poodle.n.01'),
 Synset('pug.n.01'),
 Synset('puppy.n.01'),
 Synset('spitz.n.01'),
 Synset('toy_dog.n.01'),
 Synset('working_dog.n.01')]

In [76]:
wn.synset('dog.n.01').definition()

'a member of the genus Canis (probably descended from the common wolf) that has been domesticated by man since prehistoric times; occurs in many breeds'

In [77]:
wn.synset('frump.n.01').definition()

'a dull unattractive unpleasant girl or woman'

In [78]:
wn.synset('dog.n.03').definition()

'informal term for a man'

In [79]:
dog = wn.synset('dog.n.01')
dog.hypernyms()

[Synset('canine.n.02'), Synset('domestic_animal.n.01')]

In [80]:
dog.hyponyms()

[Synset('basenji.n.01'),
 Synset('corgi.n.01'),
 Synset('cur.n.01'),
 Synset('dalmatian.n.02'),
 Synset('great_pyrenees.n.01'),
 Synset('griffon.n.02'),
 Synset('hunting_dog.n.01'),
 Synset('lapdog.n.01'),
 Synset('leonberg.n.01'),
 Synset('mexican_hairless.n.01'),
 Synset('newfoundland.n.01'),
 Synset('pooch.n.01'),
 Synset('poodle.n.01'),
 Synset('pug.n.01'),
 Synset('puppy.n.01'),
 Synset('spitz.n.01'),
 Synset('toy_dog.n.01'),
 Synset('working_dog.n.01')]

In [134]:
wn.synsets('bank')[9].hyponyms()

[Synset('vertical_bank.n.01')]

In [81]:
vocab = list(wn.all_synsets('n'))

In [82]:
len(vocab)

82115

In [188]:
_vocab = list(wn.all_synsets('v'))
vocab = vocab+_vocab

In [198]:
_vocab[:10]

[Synset('breathe.v.01'),
 Synset('respire.v.02'),
 Synset('respire.v.01'),
 Synset('choke.v.01'),
 Synset('hyperventilate.v.02'),
 Synset('hyperventilate.v.01'),
 Synset('aspirate.v.03'),
 Synset('burp.v.01'),
 Synset('force_out.v.08'),
 Synset('hiccup.v.01')]

In [189]:
len(vocab)

95882

In [ ]:
# vocab[:10]

In [191]:
vocab_ = set()
for ss in vocab:
    word = ss.name().split('.')[0]
    if '_' in word:
        continue
    vocab_.add(word)

In [192]:
len(vocab_)

44400

In [193]:
list(vocab_)[:10]

['dinky',
 'ummah',
 'volcanism',
 'sigmoidoscope',
 'newsagent',
 'watts',
 'isaac',
 'needlebush',
 'procyonid',
 'decode']

In [194]:
def get_uniques(input_list):
    uniques = set()
    for k,v in Counter(input_list).items():
        if v>1:
            continue
        uniques.add(k)
    return uniques

In [195]:
lemmatizer = WordNetLemmatizer()
stops = set(stopwords.words('english'))
word_desc = dict()
for word in vocab_:
    synsets = wn.synsets(word)
    for synset in synsets:
        txt = synset.definition().lower()
        txt = re.sub(r'\W+', ' ', txt)
        words = [w for w in txt.split() if w not in stops]
        #txts = synset.examples()
        #txt = ' '.join(txts)
        #txt = re.sub(r'\W+', ' ', txt)
        words_ = [w for w in txt.split() if w not in stops]
        words = words + words_
        hypo = synset.hyponyms()
        hypo_words = [ss.name().split('.')[0] for ss in hypo]
        hyper = synset.hypernyms()
        hyper_words = [ss.name().split('.')[0] for ss in hyper]
        _words = words+hypo_words+hyper_words
        words = list()
        for w in _words:
            words.extend(w.split('_'))
        words = [lemmatizer.lemmatize(w) for w in words]
        words = set(words)
        #add to dict
        if word in word_desc:
            word_desc[word].append(words)
        else:
            word_desc[word]=[words]

In [196]:
word2desc = dict()
for k,v in word_desc.items():
    word2desc[k] = list()
    all_words = [item for sublist in v for item in sublist]
    uniques = get_uniques(all_words)
    for words in v:
        words = set(words)
        word2desc[k].append(words&uniques)

In [197]:
word2desc['dog']

[{'animal',
  'basenji',
  'breed',
  'canine',
  'canis',
  'common',
  'corgi',
  'cur',
  'dalmatian',
  'descended',
  'dog',
  'domestic',
  'domesticated',
  'genus',
  'great',
  'griffon',
  'hairless',
  'hunting',
  'lapdog',
  'leonberg',
  'many',
  'member',
  'mexican',
  'newfoundland',
  'occurs',
  'pooch',
  'poodle',
  'prehistoric',
  'probably',
  'pug',
  'puppy',
  'pyrenees',
  'since',
  'spitz',
  'time',
  'toy',
  'wolf',
  'working'},
 {'dull', 'girl', 'unattractive', 'unpleasant', 'woman'},
 {'chap', 'informal', 'term'},
 {'morally', 'perisher', 'reprehensible', 'someone', 'villain'},
 {'beef',
  'bread',
  'minced',
  'often',
  'pork',
  'roll',
  'sausage',
  'served',
  'smoked',
  'smooth',
  'textured',
  'usually',
  'vienna'},
 {'backward',
  'fit',
  'forward',
  'hinged',
  'move',
  'moving',
  'notch',
  'prevent',
  'ratchet',
  'wheel'},
 {'fireplace', 'log', 'metal', 'support'},
 {'down', 'go', 'hound', 'intent', 'pursue', 'quest', 'run', 't

In [200]:
import sys
sys.path.append("../../imports/")
import saver as sv

In [201]:
sv.save(word2desc,"word2desc")

Saved the data


In [97]:
len(vocab_)

40968

In [118]:
list(vocab_)[:10]

['dinky',
 'ummah',
 'volcanism',
 'sigmoidoscope',
 'newsagent',
 'watts',
 'isaac',
 'needlebush',
 'procyonid',
 'shredder']

In [161]:
dog.examples()

['the dog barked all night']

179

In [143]:
for i in vocab_[100:110]:
    print(i)
    txt = i.definition().lower()
    print(txt)
    txt = re.sub(r'\W+', ' ', txt)
    print(txt)
    words = [word for word in txt.split() if word not in stops]
    print(set(words))
    word = i.name().split('.')[0]
    print(wn.synsets(word))
    print()

Synset('rally.n.02')
the feat of mustering strength for a renewed effort
the feat of mustering strength for a renewed effort
{'effort', 'mustering', 'renewed', 'strength', 'feat'}
[Synset('rally.n.01'), Synset('rally.n.02'), Synset('rally.n.03'), Synset('rally.n.04'), Synset('rally.n.05'), Synset('beat_up.v.02'), Synset('call_up.v.04'), Synset('muster.v.01'), Synset('rally.v.04'), Synset('tease.v.02')]

Synset('recovery.n.03')
the act of regaining or saving something lost (or in danger of becoming lost)
the act of regaining or saving something lost or in danger of becoming lost 
{'danger', 'something', 'saving', 'act', 'becoming', 'lost', 'regaining'}
[Synset('recovery.n.01'), Synset('convalescence.n.01'), Synset('recovery.n.03')]

Synset('running_away.n.01')
the act of leaving (without permission) the place you are expected to be
the act of leaving without permission the place you are expected to be
{'act', 'without', 'permission', 'place', 'leaving', 'expected'}
[Synset('running_away